In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
app_df = application_df.drop(columns=['EIN', 'NAME'])
app_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.

# Generate a categorical variable list (module 21-2, activity 06, cell 2)
application_cat = app_df.dtypes[app_df.dtypes == "object"].index.tolist()

# Now find the number of unique values in each column
app_df[application_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [4]:
# Count special considerations
spec_case_counts = app_df['SPECIAL_CONSIDERATIONS'].value_counts()
spec_case_counts

SPECIAL_CONSIDERATIONS
N    34272
Y       27
Name: count, dtype: int64

In [5]:
# Remove special considerations since the majority do not have special considerations
app_df = app_df.drop(columns=['SPECIAL_CONSIDERATIONS'])
app_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,5000,1


In [6]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_types = app_df['APPLICATION_TYPE'].value_counts()
app_types

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [7]:
# Adjust the cutoff application types from the venture_investments.ipynb, add T10 and T9 to 'other'

# use the variable name `application_types_to_replace`
application_types_to_replace = ['T13','T12','T2','T25','T14','T29','T15','T17','T10','T9']

# Replace in dataframe
for app in application_types_to_replace:
    app_df['APPLICATION_TYPE'] = app_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
app_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [8]:
# Keep the CLASSIFICATION column replacements the same as venture_investments.ipynb
# NOTE: excluding the value counts from the original file, only using the cutoff/replacement code

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Choose cutoff at < 1000
classifications_to_replace = ['C7000','C1700','C4000','C5000','C1270','C2700','C2800','C7100','C1300','C1280','C1230','C1400','C7200','C2300','C1240','C8000','C7120','C1500','C1800','C6000','C1250','C8200','C1238',
                              'C1278','C1235','C1237','C7210','C2400','C1720','C4100','C1257','C1600','C1260','C2710','C0','C3200','C1234','C1246','C1267','C1256','C1370','C1236','C2190','C4200','C5200','C1248',
                              'C2600','C1580','C6100','C1820','C1900','C2570','C1570','C3700','C2500','C1283','C2380','C1732','C1728','C2170','C4120','C8210','C4500','C1245','C2561','C2150']

# Replace in dataframe
for cls in classifications_to_replace:
    app_df['CLASSIFICATION'] = app_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
app_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Regenerate the categorical variable list from earlier (module 21-2, activity 06, cell 2)
application_cat = app_df.dtypes[app_df.dtypes == "object"].index.tolist()

# Now find the number of unique values in each column
app_df[application_cat].nunique()

APPLICATION_TYPE    8
AFFILIATION         6
CLASSIFICATION      6
USE_CASE            5
ORGANIZATION        4
INCOME_AMT          9
dtype: int64

In [10]:
# Convert catagorical data to numeric with pd.get_dummies
#  Help from Xpert Learning Assistant, module 21-2, activity 06

# Start with the application_cat variable from earlier
app_dummy_df = pd.get_dummies(app_df[application_cat])

# Merge the dummy df with the original app_df
app_df_encoded = app_df.merge(app_dummy_df, left_index=True, right_index=True)

# Drop the original category columns so only the encoded ones are left
app_df_encoded = app_df_encoded.drop(application_cat, axis=1)
app_df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,1,108590,1,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,1,5000,0,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
3,1,6692,1,False,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,1,142590,1,False,False,True,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [11]:
# Get list of columns to help determine y and X split
app_df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   STATUS                        34299 non-null  int64
 1   ASK_AMT                       34299 non-null  int64
 2   IS_SUCCESSFUL                 34299 non-null  int64
 3   APPLICATION_TYPE_Other        34299 non-null  bool 
 4   APPLICATION_TYPE_T19          34299 non-null  bool 
 5   APPLICATION_TYPE_T3           34299 non-null  bool 
 6   APPLICATION_TYPE_T4           34299 non-null  bool 
 7   APPLICATION_TYPE_T5           34299 non-null  bool 
 8   APPLICATION_TYPE_T6           34299 non-null  bool 
 9   APPLICATION_TYPE_T7           34299 non-null  bool 
 10  APPLICATION_TYPE_T8           34299 non-null  bool 
 11  AFFILIATION_CompanySponsored  34299 non-null  bool 
 12  AFFILIATION_Family/Parent     34299 non-null  bool 
 13  AFFILIATION_Independent       3

In [12]:
# Split our preprocessed data into our features and target arrays
# Choose y as 'IS_SUCCESSFUL' since that column is dependent on the other columns
# X will be everything else
y = app_df_encoded['IS_SUCCESSFUL'].values
X = app_df_encoded.drop(['IS_SUCCESSFUL'], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=78)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

**Original Model Evaluation**
1. 2 hidden layers with 10 nodes each.
2. Activation for hidden layers and output layer is relu.
3. 100 epochs.
4. Evaluation:

    268/268 - 0s - loss: 0.5963 - accuracy: 0.7255 - 484ms/epoch - 2ms/step

    **Loss: 0.5962861180305481, Accuracy: 0.7254810333251953**

**Optimization Attempt # 1**
1. Use the same number of hidden layers (2) and nodes (10 per layer) from original.
2. Keep activation='relu' for the hidden layers, but change the output layer activation to activation='sigmoid' since the output range is 0 or 1.
3. Try 150 epochs.


In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Used module 21-2 activity 06
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu')
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu')
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation='sigmoid')
)

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                410       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 531 (2.07 KB)
Trainable params: 531 (2.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model (module 21-2 activity 06)
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
# Train the model, use 150 epochs
fit_model = nn.fit(X_train_scaled,y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 8s 6ms/step - loss: 0.5970 - accuracy: 0.7074
Epoch 2/150
804/804 [==============================] - 3s 4ms/step - loss: 0.5632 - accuracy: 0.7245
Epoch 3/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5581 - accuracy: 0.7276
Epoch 4/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5556 - accuracy: 0.7288
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5540 - accuracy: 0.7299
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5527 - accuracy: 0.7310
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5520 - accuracy: 0.7306
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7319
Epoch 9/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5502 - accuracy: 0.7322
Epoch 10/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accura

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5544 - accuracy: 0.7263 - 469ms/epoch - 2ms/step
Loss: 0.5544385313987732, Accuracy: 0.7262973785400391


Optimization attempt # 1 yielded about the same results as the original model, but did not hit the 75% accuracy threshold requirement.

**Optimization Attempt # 2**
1. Keep 2 hidden layers, with the first layer having 10 nodes. DECREASE the second layer to 8 nodes.
2. Keep activations the same as attempt # 1.
3. Keep 150 epochs.

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Used module 21-2 activity 06
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu')
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu')
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation='sigmoid')
)

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                410       
                                                                 
 dense_4 (Dense)             (None, 8)                 88        
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 507 (1.98 KB)
Trainable params: 507 (1.98 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Compile the model (module 21-2 activity 06)
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# Train the model, use 150 epochs
fit_model = nn.fit(X_train_scaled,y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 0.6172 - accuracy: 0.6962
Epoch 2/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5672 - accuracy: 0.7242
Epoch 3/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5607 - accuracy: 0.7269
Epoch 4/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5578 - accuracy: 0.7283
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5550 - accuracy: 0.7304
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5536 - accuracy: 0.7318
Epoch 7/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5529 - accuracy: 0.7315
Epoch 8/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7314
Epoch 9/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5513 - accuracy: 0.7322
Epoch 10/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5507 - accura

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5531 - accuracy: 0.7254 - 693ms/epoch - 3ms/step
Loss: 0.5530616044998169, Accuracy: 0.7253644466400146


Optimization attempt # 2 did slightly worse than attempt # 1, with valuation values closer to the original model.

**Optimization Attempt # 3**
Use auto-optimization kerastuner.

In [22]:
# Install kerastuner
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.0 MB/s eta 0:00:00


In [23]:
# Using code from module 21-2 activity 04, create a method that creates a new Sequential model with hyperparameter options
# Also used Xpert Learning Assistant to help explain elements of the code.

# Define the method, where hp is hyperparameter options
def create_model(hp):
  nn_model = tf.keras.models.Sequential()

  # Allow kerastuner to decide which activation function to use in hidden layers. Give it the options of relu, tanh, and sigmoid
  activation = hp.Choice('activation', ['relu','tanh','sigmoid'])

  # Allow kerastuner to choose the number of neurons in the first layer.
  # First run: Since there are 41 columns (including the inputs and outputs), set the max at 41.
  # First run: Use step=4 so granularity isn't too fine, but isn't too coarse. 4 is approximately 10% of 41.
  # Second run: reduced max to 20, left step at 4.
  nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
                                                  min_value=1,
                                                  #max_value=41,
                                                  max_value=20,
                                                  step=4), activation=activation, input_dim=len(X_train[0])))

  # Allow kerastuner to choose the number of hidden layers and the number of neurons in hidden layers.
  # Choose a limit of 5 hidden layers for kerastuner to select from.
  # First run: Choose a limit of 15 neurons for kerastuner to select from.
  # Second run: Choose a limit of 10 neurons.
  # Use step=3.
  for i in range(hp.Int('num_layers', 1,5)):
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_'+str(i),
                                                    min_value=1,
                                                    #max_value=15,
                                                    max_value=10,
                                                    step=3),
                                       activation=activation))

  nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

  # Compile the model
  nn_model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

  return nn_model

In [24]:
# Import the kerastuner library
import keras_tuner as kt

# Choose hyperband_iterations=2 to run the tuning process 2 times
# First run: max 100 epochs.
# Second run: max 20 epochs.
tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=20,
    hyperband_iterations=2
)

In [25]:
# Run the kerastuner to search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 23s]
val_accuracy: 0.7274635434150696

Best val_accuracy So Far: 0.7301457524299622
Total elapsed time: 00h 24m 53s


In [26]:
# Export the model and the kerastuner model to HDF5.
nn.save("/content/drive/MyDrive/DataClassNotebooks/deep-learning-challenge/AlphabetSoupCharity_Optimization.h5")

# Tried to save the auto-optimizer model as well but it did not work.
# nn_model.save("/content/drive/MyDrive/DataClassNotebooks/deep-learning-challenge/AlphabetSoupCharity_Auto_Optimization.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
